<a href="https://colab.research.google.com/github/sajal1302/Study-Material/blob/master/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pyspark

     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 47.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=96a7e31d3bdebeb3b3abcaa260d1b255b0950a4a621568c017559f57273f9a13
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession

In [101]:
from pyspark.sql.functions import *
import logging
logger = logging.getLogger('BCG-Use-Case')
logger.setLevel(logging.INFO)

In [3]:
def get_spark_session():
    spark = SparkSession.builder.appName("BCG-Project").enableHiveSupport().getOrCreate()
    return spark

In [5]:
config = { 'files' : ['Charges_use.csv' ,
                      'Damages_use.csv' ,
                      'Endorse_use.csv',
                      'Primary_Person_use.csv' ,
                      'Restrict_use.csv' ,
                      'Units_use.csv'] ,
          'files_loc' : '',
          
          'feeds':{
              'Charges_use': {
                  'feed_name' : 'Charges_use.csv',
                  'feed_location' : '',
                  'delimeter' : '',
                  'compression' : '',
                  
              },
              'Damages_use': {
                  'feed_name' : 'Damages_use.csv',
                  'feed_location' : '',
                  'delimeter' : '',
                  'compression' : '',
                  
              },
              'Endorse_use': {
                  'feed_name' : 'Damages_use.csv',
                  'feed_location' : '',
                  'delimeter' : '',
                  'compression' : '',
                  
              },
              'Primary_Person_use': {
                  'feed_name' : 'Primary_Person_use.csv',
                  'feed_location' : '',
                  'delimeter' : '',
                  'compression' : '',
                  
              },
              'Restrict_use': {
                  'feed_name' : 'Restrict_use.csv',
                  'feed_location' : '',
                  'delimeter' : '',
                  'compression' : '',
                  
              },
              'Units_use': {
                  'feed_name' : 'Units_use.csv',
                  'feed_location' : '',
                  'delimeter' : '',
                  'compression' : '',
                  
              }
          }  
    
}

In [6]:
spark = get_spark_session()

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
spark

In [15]:
! ls drive/MyDrive/bcg/staging_files/

Charges_use.csv  Endorse_use.csv	 Restrict_use.csv
Damages_use.csv  Primary_Person_use.csv  Units_use.csv


In [20]:
! ls bcg/config

config.json


In [21]:
import json

In [28]:
def load_json(file_path):
  with open(file_path) as f:
    config = json.load(f)
    return config

In [103]:
bcg_conf = load_json('bcg/config/config.json')

In [104]:
def load_file(bcg_conf , entity_name):
  try :
    feed_name = bcg_conf['feeds'][entity_name]['feed_name']
    file_loc = bcg_conf['feeds'][entity_name]['feed_location']
    file_format = bcg_conf['feeds'][entity_name]['file_format']
    delimeter = bcg_conf['feeds'][entity_name]['delimeter']
    header = bcg_conf['feeds'][entity_name]['header']
    full_path = file_loc + feed_name

    print(feed_name , file_loc , file_format, full_path )
    df_ = spark.read.load(full_path, 
                    format= file_format, sep= delimeter, inferSchema="true",
                    header=header)
    return df_

  except:
    logger.error("Check the Config for the Entity : {} ".format(entity_name))

In [105]:
charges_use_df = load_file(bcg_conf , 'Charges_use')

Charges_use.csv /content/drive/MyDrive/bcg/staging_files/ csv /content/drive/MyDrive/bcg/staging_files/Charges_use.csv


In [115]:
def trim_columns(df):
  df_ = df.select([ trim(col(colm)).alias(colm) if dtype =='string' else col(colm) for colm,dtype in df.dtypes ])
  return df_

In [114]:
charges_use_df = trim_columns(charges_use_df)

In [108]:
charges_use_df.show(10, False)

+--------+--------+--------+----------------------------------------+------------+
|CRASH_ID|UNIT_NBR|PRSN_NBR|CHARGE                                  |CITATION_NBR|
+--------+--------+--------+----------------------------------------+------------+
|14768622|1       |1       |DRIVING WHILE INTOXICATED               |null        |
|14838637|1       |1       |DWI                                     |1600000015  |
|14838641|1       |1       |RAN RED LIGHT SOLID (TC 544.007)        |L20440      |
|14838641|2       |1       |NO DRIVER'S LICENSE (TC521.025)         |L23141      |
|14838668|1       |1       |DRIVING WHILE INTOXICATED               |TX4IC50SRJD3|
|14838669|2       |1       |DWI W/BAC >.015                         |2015-000006 |
|14838670|1       |1       |DRIVING WHILE INTOXICATED               |2016-000003 |
|14838685|1       |1       |FAILED TO DRIVE SINGLE LANE, NOL, NO INS|138434825   |
|14838693|1       |1       |DRIVING WHILE INTOXICATED               |TX4IC60UKQND|
|148

In [116]:
damages_use_df = load_file(bcg_conf , 'Damages_use')


Damages_use.csv /content/drive/MyDrive/bcg/staging_files/ csv /content/drive/MyDrive/bcg/staging_files/Damages_use.csv


In [118]:
charges_use_df = trim_columns(load_file(bcg_conf , 'Charges_use'))
damages_use_df = trim_columns(load_file(bcg_conf , 'Damages_use'))
endorse_use_df = trim_columns(load_file(bcg_conf , 'Endorse_use'))
primary_person_use_df = trim_columns(load_file(bcg_conf , 'Primary_Person_use'))
restrict_use_df = trim_columns(load_file(bcg_conf , 'Restrict_use'))
units_use_df = trim_columns(load_file(bcg_conf , 'Units_use'))

Charges_use.csv /content/drive/MyDrive/bcg/staging_files/ csv /content/drive/MyDrive/bcg/staging_files/Charges_use.csv
Damages_use.csv /content/drive/MyDrive/bcg/staging_files/ csv /content/drive/MyDrive/bcg/staging_files/Damages_use.csv
Damages_use.csv /content/drive/MyDrive/bcg/staging_files/ csv /content/drive/MyDrive/bcg/staging_files/Damages_use.csv
Primary_Person_use.csv /content/drive/MyDrive/bcg/staging_files/ csv /content/drive/MyDrive/bcg/staging_files/Primary_Person_use.csv
Restrict_use.csv /content/drive/MyDrive/bcg/staging_files/ csv /content/drive/MyDrive/bcg/staging_files/Restrict_use.csv
Units_use.csv /content/drive/MyDrive/bcg/staging_files/ csv /content/drive/MyDrive/bcg/staging_files/Units_use.csv
